In [41]:
import gensim
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
stemmer = SnowballStemmer('english')
import numpy as np
np.random.seed(2018)
import nltk
import logging
import os
import requests as re
import pandas as pd
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from pprint import pprint
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from wordcloud import WordCloud
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/kevins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/kevins/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
papers = pd.read_csv(os.getcwd().replace('scripts', 'data/papers/final_valid_papers.csv'))



In [3]:
stop_words = stopwords.words('english')


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [5]:
keys = [
        "e3fa6eb97964b552119ea5fc78093789",
"c7cdff212e1739790e728a85953e58d",
"c515ad531ff5767f6b85a20e2c190ca9",
"ef2e9e7f3bda7235f0c079cb38fd2a9a",
"f61a0ca8673900c9539e5b3228c95a51"]

In [6]:
papers


,journal_id,volume_id,year,link,doi,date,pii,eid,scopus_id,citations,authors,affiliations,open_access
0,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104474,2023-08-01,S0309170823001094,2-s2.0-85161258968,8.516126e+10,0.0,"58307014400, 36008901600, 26635952600",60031268:60031268:60007989,True
1,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104475,2023-08-01,S0309170823001100,2-s2.0-85161636300,8.516164e+10,0.0,"53984251100, 35610240400","60104134:60104134, 60022461",True
2,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104476,2023-08-01,S0309170823001112,2-s2.0-85161667219,8.516167e+10,0.0,25822520800,60025063,True
3,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104477,2023-08-01,S0309170823001124,2-s2.0-85161661769,8.516166e+10,0.0,"57210424267, 36160282300, 7404296426",60018394:60018394:60018394,True
4,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104485,2023-08-01,S0309170823001203,2-s2.0-85161678744,8.516168e+10,0.0,"56708113100, 7202660185",60030447:60007566,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26469,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.020,2018-01-01,S1364815217304140,2-s2.0-85032268729,8.503227e+10,142.0,"34571084000, 55188785600, 33767605600, 5721638...","60016437:60012197:60012197:60012197, 60025278:...",True
26470,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.010,2018-01-01,S1364815216304005,2-s2.0-85032353211,8.503235e+10,54.0,"37111876100, 15069126500, 57189996027, 3574567...","60013141:60025182:60025182:60025182:60013141, ...",True
26471,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.08.002,2018-01-01,S1364815216309550,2-s2.0-85032684995,8.503268e+10,5.0,"7006615627, 49661600900, 15132591800, 66033150...","60021439:60030486, 109682148:60021439:60021439...",True
26472,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.011,2018-01-01,S136481521630843X,2-s2.0-85032736133,8.503274e+10,45.0,"56111768300, 37010006200, 43361846100, 5611302...",60068861:60020071:60068861:60024695:60102538:6...,True


In [15]:
#get the full text
def fullText(i):
    doi = papers.iat[i,4]
#     done = False
#     print(doi)
    try:
        js = re.get(
            f"https://api.elsevier.com/content/article/doi/{doi}",
            headers = {"X-ELS-APIKey":"a06b94f351d042ccec3a28009009547d", "Accept":"application/json"}

,
        )
        
        r = js.json()
        
#         print(r)

        full_text = r['full-text-retrieval-response']['originalText']
#         full_text = re.sub(r'^https?:\/\/.[^\s]+', '', full_text)
    
        
        abstr = r['full-text-retrieval-response']['coredata']['dc:description'][:25]
        
        index = full_text.find(abstr)
        if(index==-1):
            other.append(i)
            return None
        full_text = full_text[index:]
        
        
        full_text = ''.join(full_text.split('References')[:-1])
        
        print(full_text)

        full_text = full_text.split(" ")
        full_text_words = list(sent_to_words(full_text))
        # remove stop words
        full_text_words = remove_stopwords(full_text_words)
        
        return full_text_words
    except Exception as e:
        print(str(e))
        if js.status_code == 404:
            no_data.append(i)
        elif js.status_code ==429:
            print("Used up " +str(e))
        

        return None
    



In [16]:
paper_words =fullText(100)

In this paper, we report on experimental and computational studies investigating the evolution of pore structure and permeability of a microporous carbonate rock during chemical dissolution using information provided from X-ray micro-computed tomography (µ-CT) and mercury intrusion porosimetry (MIP) techniques. We consider a chemical dissolution in a core sample by a nonacidic solution wherein a quasi-uniform modification of pore structure occurred. First, we conduct a comparative analysis on the capabilities and limitations of the µ-CT and MIP methods to quantify the evolution of pore size and pore surface area. In particular, we incorporate micropores into the calculations of the pore size-related parameters to highlight the uncertainties that ignoring microporosity causes. In the second part of the paper, we present predictive results on the fractional changes in the permeability of the rock using the Katz-Thompson (KT) and Kozeny-Carman (KC) models. The predictions are based on two

In [15]:
paper_words

'although the fluid invasion process impacts the transport and dispersion of chemicals and contaminants in the resident phase of natural manufactured and biological unsaturated porous media these effects remain poorly understood in this study we investigate the role of hydrodynamic dispersion in different invasion patterns within the resident phase while nonidealities in pore structures contribute to mechanical dispersion our findings demonstrate that the interface front s morphology and the invading phase s distribution significantly influence mechanical dispersion within the resident phase our results show that the complex distribution of the invading phase under unstable displacement greatly affects mechanical dispersion measures such as velocity distribution variance and mean square displacement obtained through particle tracking the interface also alters hydrodynamic dispersion measures such as the local peclet number and generates extensive diffusion dominated regions across the 

In [11]:
full_text_df = pd.read_csv('full_text_df.csv')

In [12]:
full_text_df

,index,text
0,0,although the fluid invasion process impacts th...
1,1,in order to investigate the migration of nanop...
2,2,in august 2019 this journal published the pape...
3,3,reservoir management and contaminant transport...
4,4,this study introduces firm information gain fo...
5,5,watersheds act as low pass filters damping and...
6,6,in dam break problems two dimensional 2d model...
7,7,a hydrological model incurs three types of unc...
8,8,per and polyfluoroalkyl substances pfass have ...
9,9,interactions between the tide and sloping sea ...


In [23]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [30]:
processed_docs = full_text_df['text'].map(preprocess)

In [31]:
id2word = corpora.Dictionary(processed_docs)

In [34]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [35]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [61]:
processed_docs[0]

['fluid',
 'invas',
 'process',
 'impact',
 'transport',
 'dispers',
 'chemic',
 'contamin',
 'resid',
 'phase',
 'natur',
 'manufactur',
 'biolog',
 'unsatur',
 'porous',
 'media',
 'effect',
 'remain',
 'poor',
 'understand',
 'studi',
 'investig',
 'role',
 'hydrodynam',
 'dispers',
 'differ',
 'invas',
 'pattern',
 'resid',
 'phase',
 'nonid',
 'pore',
 'structur',
 'contribut',
 'mechan',
 'dispers',
 'find',
 'demonstr',
 'interfac',
 'morpholog',
 'invad',
 'phase',
 'distribut',
 'signific',
 'influenc',
 'mechan',
 'dispers',
 'resid',
 'phase',
 'result',
 'complex',
 'distribut',
 'invad',
 'phase',
 'unstabl',
 'displac',
 'great',
 'affect',
 'mechan',
 'dispers',
 'measur',
 'veloc',
 'distribut',
 'varianc',
 'mean',
 'squar',
 'displac',
 'obtain',
 'particl',
 'track',
 'interfac',
 'alter',
 'hydrodynam',
 'dispers',
 'measur',
 'local',
 'peclet',
 'number',
 'generat',
 'extens',
 'diffus',
 'domin',
 'region',
 'domain',
 'ultim',
 'lead',
 'solut',
 'trap',
 'pore

In [49]:
# number of topics
num_topics = 4
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.023*"model" + 0.012*"flow" + 0.009*"time" + 0.009*"fractur" + '
  '0.008*"particl" + 0.007*"valu" + 0.007*"error" + 0.007*"data" + '
  '0.007*"water" + 0.007*"effect"'),
 (1,
  '0.019*"model" + 0.011*"valu" + 0.011*"time" + 0.009*"water" + '
  '0.009*"method" + 0.009*"data" + 0.008*"streamflow" + 0.008*"error" + '
  '0.007*"rainfal" + 0.007*"measur"'),
 (2,
  '0.030*"model" + 0.009*"time" + 0.009*"method" + 0.008*"fractur" + '
  '0.007*"distribut" + 0.007*"flow" + 0.007*"water" + 0.007*"simul" + '
  '0.007*"valu" + 0.007*"data"'),
 (3,
  '0.017*"model" + 0.012*"time" + 0.011*"flow" + 0.010*"water" + 0.008*"simul" '
  '+ 0.007*"valu" + 0.006*"particl" + 0.006*"pfas" + 0.006*"method" + '
  '0.005*"increas"')]


In [43]:
pyLDAvis.enable_notebook()

In [44]:
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [50]:
#bag of words
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.016174 -0.000528       1        1  33.109939
2     -0.001121 -0.007583       2        1  27.999977
3     -0.002430  0.010050       3        1  21.602035
0     -0.012622 -0.001939       4        1  17.288049, topic_info=         Term        Freq       Total Category  logprob  loglift
385     model  825.000000  825.000000  Default  30.0000  30.0000
233      flow  310.000000  310.000000  Default  29.0000  29.0000
1124  fractur  220.000000  220.000000  Default  28.0000  28.0000
438   particl  192.000000  192.000000  Default  27.0000  27.0000
663    vertic   83.000000   83.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
295    inform   25.078401  131.162002   Topic4  -5.5447   0.1007
289   increas   29.405297  205.164373   Topic4  -5.3855  -0.1875
415    number   26.131627  148.648319   Topic4  -5.5036   0.0167
2555     pfas   26.148683  226.062110   Topic4  -5.5029  -0.4019
368    measur   25.430407  190.110534   Topic4  -5.5308  -0.2565

[330 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2236      1  0.466439  accept
2236      2  0.186576  accept
2236      3  0.186576  accept
2236      4  0.093288  accept
694       1  0.184782   adopt
...     ...       ...     ...
2233      4  0.367982    xiao
683       1  0.262059    zone
683       2  0.249580    zone
683       3  0.299496    zone
683       4  0.187185    zone

[872 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 1])

In [54]:
#TF IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"pump" + 0.001*"gain" + 0.001*"well" + 0.001*"firm" + 0.001*"discrimin" + 0.001*"probabl" + 0.001*"design" + 0.001*"covari" + 0.001*"entropi" + 0.001*"maxim"
Topic: 1 Word: 0.001*"watert" + 0.001*"sorptiv" + 0.001*"hybrid" + 0.001*"wave" + 0.001*"solver" + 0.001*"disput" + 0.001*"moistur" + 0.001*"break" + 0.001*"coastal" + 0.001*"unconfin"
Topic: 2 Word: 0.002*"pfas" + 0.001*"loa" + 0.001*"streamflow" + 0.001*"rainfal" + 0.001*"uncertainti" + 0.001*"watersh" + 0.001*"runoff" + 0.001*"tree" + 0.001*"event" + 0.001*"predictor"
Topic: 3 Word: 0.002*"fractur" + 0.001*"invas" + 0.001*"nanoparticl" + 0.001*"storag" + 0.001*"filter" + 0.001*"signal" + 0.001*"aggreg" + 0.001*"attach" + 0.001*"resid" + 0.001*"genet"


In [55]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.0024933753967470423),
 (1, 0.005789438384807881),
 (2, 0.008282813781554925),
 (3, 0.01299272183247038),
 (4, 0.0016053751826275932),
 (5, 0.0012830240889597827),
 (6, 0.003296062988060839),
 (8, 0.009888188964182516),
 (9, 0.021654536387450633),
 (10, 0.026368503904486713),
 (12, 0.0018375318807430843),
 (13, 0.0289471919240394),
 (14, 0.009888188964182516),
 (15, 0.0024933753967470423),
 (16, 0.010264192711678262),
 (17, 0.014960252380482254),
 (18, 0.0043309072774901265),
 (19, 0.008282813781554925),
 (20, 0.005789438384807881),
 (21, 0.0003790007734253294),
 (23, 0.0003790007734253294),
 (25, 0.0016053751826275932),
 (26, 0.0024933753967470423),
 (27, 0.004816125547882779),
 (28, 0.005789438384807881),
 (29, 0.005789438384807881),
 (30, 0.0024933753967470423),
 (31, 0.008282813781554925),
 (32, 0.01299272183247038),
 (33, 0.007350127522972337),
 (35, 0.0043309072774901265),
 (36, 0.0043309072774901265),
 (37, 0.005789438384807881),
 (39, 0.0008026875913137966),
 (42, 0.02484

In [56]:
#tfidf
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.001442 -0.000577       1        1  36.211945
2     -0.000073  0.000945       2        1  29.609960
1     -0.001235 -0.000773       3        1  24.324185
0     -0.000133  0.000405       4        1   9.853910, topic_info=           Term       Freq      Total Category  logprob  loglift
2555       pfas  28.000000  28.000000  Default  30.0000  30.0000
2661     watert  23.000000  23.000000  Default  29.0000  29.0000
1452       pump  16.000000  16.000000  Default  28.0000  28.0000
250        gain  16.000000  16.000000  Default  27.0000  27.0000
1535       well  17.000000  17.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
996    sequenti   1.560795  13.839412   Topic4  -7.7594   0.1350
281    identifi   1.618860  14.430222   Topic4  -7.7228   0.1297
365      matrix   1.662753  15.004091   Topic4  -7.6961   0.1175
1653      layer   1.697975  15.487352   Topic4  -7.6751   0.1067
861   groundwat   1.652810  15.000727   Topic4  -7.7021   0.1117

[167 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1065      1  0.309914  absorpt
1065      2  0.247931  absorpt
1065      3  0.371896  absorpt
1065      4  0.061983  absorpt
1237      1  0.340463      add
...     ...       ...      ...
1233      4  0.062854     zhao
2604      1  0.335709     θave
2604      2  0.335709     θave
2604      3  0.201425     θave
2604      4  0.067142     θave

[528 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

In [85]:
threshold = 25

In [86]:
for i in range(26474):
    tmp = buzzwords_in_papers[buzzwords_in_papers['paper_id']==i]
    
    grouped = tmp.groupby('buzz_id').count()
    
#     print(grouped)
    
    for j in range(grouped.shape[0]):
        if grouped.iat[j,0] >=threshold:
            lots_of_words.loc[len(lots_of_words.index)] = [i, grouped.iloc[j].name, grouped.iat[j,0]]

In [74]:
lots_of_words

,paper_id,buzz_id,count
0,1,18,102
1,3,6,65
2,7,6,59
3,9,37,62
4,19,0,99
...,...,...,...
3441,26466,24,51
3442,26468,10,75
3443,26470,0,90
3444,26470,6,83


In [76]:
len(set(lots_of_words['paper_id'].tolist()))

2225